# **Install / Import / Read Data :**

In [1]:
!pip install pytorch-tabnet

In [2]:
import pandas as pd 
import numpy as np
from google.colab import drive
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.model_selection import *
drive.mount("/content/drive")
path = "/content/drive/My Drive/MachineHack/Anomalies_Detection/"


train_data                          = pd.read_csv(path+"Train.csv")
test_data                           = pd.read_csv(path+"Test.csv")
train_y                             = train_data.Class.values
train_data                          = train_data.drop(['Class'],axis=1)    


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Eliminating features with low sum of ones :

In [3]:
columns         = train_data.columns
num_cols        = ['feature_1','feature_2','feature_3']
cat_cols        = list(set(columns)-set(num_cols))
needed_cat_cols  = []
for cols in cat_cols:
    if (train_data[cols].sum()>0):
        needed_cat_cols.append(cols)
        
train_df         = train_data[needed_cat_cols+num_cols]
test_df          = test_data[needed_cat_cols+num_cols]
concat_df        = pd.concat((train_df,test_df),axis=0)
print(train_df.shape,test_df.shape)




(1763, 1522) (756, 1522)


In [4]:
train            = train_df.values
test             = test_df.values

# **Modeling :**

In [5]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [6]:
oof_pred               = np.zeros((len(train),))
y_pred_final           = np.zeros((len(test), ))
num_models             = 1

n_splits               = 15
error                  = []
kf                     = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (tr_ind, val_ind) in enumerate(kf.split(train, train_y)):
    
    
    
    X_train, X_val     = train[tr_ind], train[val_ind]
    y_train, y_val     = train_y[tr_ind], train_y[val_ind]
    


    clf = TabNetClassifier(
                            n_d=64, n_a=64, n_steps=5,
                            gamma=1.5, n_independent=2, n_shared=2,
                            
                            lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
                            optimizer_fn=torch.optim.AdamW,
                            optimizer_params=dict(lr=5e-2),
                            scheduler_params = {"gamma": 0.95,
                                            "step_size": 20},
                            scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                           seed = 1997
                        )

    
    np.random.seed(seed=42)
    clf.fit(
                X_train=X_train, y_train=y_train,
                X_valid=X_val, y_valid=y_val,
                max_epochs=25, patience=100,
                batch_size=32, virtual_batch_size=16,
                
            )

    val_pred1          = clf.predict_proba(X_val)[:,1]
#     auc_roc.append(roc_auc_score(y_val,val_pred1))
    print('validation roc_auc model 1 fold-',fold+1,': ',roc_auc_score(y_val,val_pred1))
    
    
    val_pred           = val_pred1
    #print('validation ROC_AUC fold-',fold+1,': ',roc_auc_score(y_val, val_pred))
    
    oof_pred[val_ind]  = val_pred
    
    y_pred_final += (clf.predict_proba(test)[:,1])/(n_splits)
    
#     print('\n')
    
print('OOF ROC_AUC:- ',(roc_auc_score(train_y,oof_pred)))

Device used : cuda
Will train until validation stopping metric hasn't improved in 100 rounds.
---------------------------------------
| EPOCH |  train  |   valid  | total time (s)
| 1     | 0.50134 |  0.52500 |   2.7       
| 2     | 0.61021 |  0.75602 |   5.2       
| 3     | 0.62456 |  0.70278 |   7.4       
| 4     | 0.65965 |  0.66343 |   9.6       
| 5     | 0.68952 |  0.62546 |   11.9      
| 6     | 0.69841 |  0.85602 |   14.1      
| 7     | 0.76796 |  0.79676 |   16.4      
| 8     | 0.72611 |  0.75880 |   18.8      
| 9     | 0.80771 |  0.71528 |   21.1      
| 10    | 0.84913 |  0.86667 |   23.6      
| 11    | 0.86328 |  0.79676 |   25.8      
| 12    | 0.85541 |  0.88935 |   28.1      
| 13    | 0.88129 |  0.79583 |   30.4      
| 14    | 0.84872 |  0.90463 |   32.7      
| 15    | 0.87974 |  0.93241 |   34.9      
| 16    | 0.90699 |  0.87731 |   37.0      
| 17    | 0.90822 |  0.87361 |   39.1      
| 18    | 0.91235 |  0.94491 |   41.3      
| 19    | 0.90763 |  0.85370

# **Submission :**

In [10]:
submission_df       = pd.read_csv("/content/drive/My Drive/MachineHack/Anomalies_Detection/Sample_submission.csv")
submission_df['Class'] = y_pred_final

In [11]:
submission_df.to_csv('tabnet.csv',index=False)

In [12]:
submission_df['Class'].mean()

0.08726631907116354

In [13]:
submission_df.head()

,Class
0,0.310625
1,0.038028
2,0.072966
3,0.319686
4,0.417845
